In [1]:
#import tensorflow as tf
#import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import os
import json
import zipfile

In [3]:
!pip install spacy

In [4]:
pip install googletrans==4.0.0-rc1

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 40.0 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 5.9 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17397 sha256=f0513a0045412476aa6f75fe4b9bc0ea207b20e97b72bbf566aa27d1d9bad6c8
  Stored in directory: /root/.cache/pip/wheels/c0/59/9f/7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
Successfully built googletrans
  Attempting uninstall: h11
    Found existing installation: h11 0.14.0
    Uninstalling h11-

In [6]:
df_raw = pd.read_csv('/kaggle/input/recipe-ds/RAW_recipes.csv')

In [7]:
def check_ingredients_in_steps(ingredients, steps):
    for ingredient in ingredients:
        if ingredient not in ' '.join(steps):
            return False
    return True


In [8]:
df_raw['all_ingredients_in_steps'] = df_raw.apply(lambda row: check_ingredients_in_steps(row['ingredients'], row['steps']), axis=1)
count = df_raw['all_ingredients_in_steps'].sum()
print(f'Número de registros en los que todos los ingredientes están incluidos en los pasos: {count}')

Número de registros en los que todos los ingredientes están incluidos en los pasos: 150793


In [9]:
#Filtramos el dataset para quedarnos con aqellas recetas que incluyen todos los ingredientes
filtered_df = df_raw[df_raw['all_ingredients_in_steps']]

In [10]:
pd.set_option('display.max_colwidth', None)
filtered_df['ingredients'][:1]

0    ['winter squash', 'mexican seasoning', 'mixed spice', 'honey', 'butter', 'olive oil', 'salt']
Name: ingredients, dtype: object

## Obtener los ingredientes diferentes

In [12]:
import pandas as pd

# Supongamos que 'df' es tu DataFrame con los datos y 'ingredients' es la columna de ingredientes.

# Convierte la columna 'ingredients' en una lista de listas.
ingredientes_lista = filtered_df['ingredients'].str.split(',')

# Usa una comprensión de listas para aplanar la lista de listas.
ingredientes = [ingrediente.strip() for sublist in ingredientes_lista for ingrediente in sublist]

# Crea un conjunto (set) para eliminar duplicados y luego conviértelo de nuevo en una lista.
ingredientes_unicos = list(set(ingredientes))

# Cuenta la cantidad de ingredientes diferentes.
cantidad_ingredientes_diferentes = len(ingredientes_unicos)

# Imprime la lista de ingredientes únicos y la cantidad.
#print("Ingredientes Únicos:")
#for ingrediente in ingredientes_unicos:
#    print(ingrediente)

print("\nCantidad de Ingredientes Diferentes:", cantidad_ingredientes_diferentes)


Cantidad de Ingredientes Diferentes: 24606


In [34]:
import pandas as pd
import re
from collections import Counter

# Supongamos que 'df' es tu DataFrame con los datos y 'ingredients' es la columna de ingredientes.

# Convierte la columna 'ingredients' en una lista de listas.
ingredientes_lista = filtered_df['ingredients'].str.split(',')



import re

def limpiar_ingrediente(ingrediente):
    # Eliminar caracteres no deseados (por ejemplo, comillas y otros caracteres especiales).
    ingrediente = re.sub(r"[^\w\s-]", "", ingrediente)
    
    # Convertir a minúsculas.
    ingrediente = ingrediente.lower()
    
    # Si hay un guion entre dos letras o números, reemplazarlo por un espacio.
    ingrediente = re.sub(r'(?<=[a-zA-Z0-9])-(?=[a-zA-Z0-9])', ' ', ingrediente)
    
    return ingrediente


# Convierte la lista de listas en una lista plana y limpia los ingredientes.
ingredientes_unicos_limpios = [limpiar_ingrediente(ingrediente.strip()) for sublist in ingredientes_lista for ingrediente in sublist]

# Usa Counter para contar la frecuencia de cada ingrediente limpio.
ingredient_frequency = Counter(ingredientes_unicos_limpios)

# Convierte el resultado en un DataFrame.
ingredient_frequency_df = pd.DataFrame(ingredient_frequency.items(), columns=['Ingrediente', 'Frecuencia'])

# Ordena el DataFrame por frecuencia descendente.
ingredient_frequency_df = ingredient_frequency_df.sort_values(by='Frecuencia', ascending=False)

# Ahora, ingredient_frequency_df contiene la frecuencia de ingredientes limpios en el DataFrame original.
print(ingredient_frequency_df)


                               Ingrediente  Frecuencia
6                                     salt       59464
4                                   butter       39850
36                                   sugar       31794
56                                   onion       25368
20                                   water       24416
...                                    ...         ...
9551   feta cheese with rosemary and basil           1
9552                              gai choy           1
9558                          potato peels           1
9559                     10 inch pie shell           1
12720                            nepitella           1

[12721 rows x 2 columns]


los ingredientes unicos despues de la limpieza se han quedado en 12721

In [16]:
ingredient_frequency_df.tail(200)

,Ingrediente,Frecuencia
10390,kangkong,1
10391,kangaroo steak,1
6455,holland rusks,1
10394,capsicum salsa,1
10396,new york strip steak,1
...,...,...
10301,soy grits,1
6603,bubblegum,1
6602,chickle,1
10305,powdered jaggery,1


In [15]:
ingredient_frequency_df.to_json('ingredient_todos.json', orient='records')

## Contamos cuantas veces tenemos un ingrediente comun en el dataset que tiene las frecuencias y los ingredientes limpios de caracteres

In [45]:
palabra_clave = 'onion'
ingredientes_con_palabra_clave = [ingrediente for ingrediente in ingredient_frequency_df['Ingrediente'] if palabra_clave in ingrediente]
print("Cantidad de ingredientes que incluyen la palabra clave", len(ingredientes_con_palabra_clave))
print(ingredientes_con_palabra_clave)

Cantidad de ingredientes que incluyen la palabra clave 177
['onion', 'onions', 'green onions', 'green onion', 'red onion', 'onion powder', 'yellow onion', 'white onion', 'sweet onion', 'red onions', 'yellow onions', 'spring onions', 'onion soup mix', 'dried onion flakes', 'diced onion', 'onion salt', 'dry onion soup mix', 'sweet onions', 'white onions', 'spring onion', 'vidalia onion', 'spanish onion', 'vidalia onions', 'dried onion', 'lipton onion soup mix', 'instant minced onion', 'brown onion', 'onion flakes', 'white pearl onions', 'purple onion', 'french fried onions', 'diced onions', 'french onion soup', 'green onion top', 'simply potatoes diced potatoes with onion', 'spanish onions', 'white pearl onion', 'brown onions', 'small onion', 'green onions with tops', 'fried onions', 'green onion tops', 'yellow sweet onion', 'french onion soup mix', 'french fried onion rings', 'condensed french onion soup', 'fresh onion', 'granulated onion', 'french onion dip', 'cooking onion', 'baby oni

Observamos en el resultado previo que los ingredientes se repiten en version  singular y plural con palabras comunes, etc

**Vamos a adicionar dos columnas a nuestro dataset que ya tiene los ingredientes unicos y la frecuencia de uso
una columna Lematizado, que contiene el ingrediente lematizado
y una columna con la traduccion al español de dicho ingrediente**

In [40]:
import spacy

# Cargar el modelo de spaCy para el idioma inglés
nlp = spacy.load("en_core_web_sm")

# Función para limpiar y lematizar un ingrediente
def limpiar_y_lematizar(ingrediente):
    # Aplicar el modelo de spaCy al ingrediente
    doc = nlp(ingrediente)
    
    # Obtener la forma base (lemma) de cada token y unirlos de nuevo en una cadena
    lematizado = " ".join([token.lemma_ for token in doc])
    
    return lematizado

In [41]:
from googletrans import Translator
translator = Translator()

# Función para obtener la traducción en español.
def obtener_traduccion(lematizado):
    try:
        traduccion = translator.translate(lematizado, src='en', dest='es')
        return traduccion.text
    except Exception as e:
        return 'Error de traducción'

In [42]:
# Agregar una columna con el ingrediente lematizado
ingredient_frequency_df['Lematizado'] = ingredient_frequency_df['Ingrediente'].apply(limpiar_y_lematizar)

# Agregar una columna con la traducción
ingredient_frequency_df['Traducción'] = ingredient_frequency_df['Lematizado'].apply(obtener_traduccion)

# Ahora, ingredient_frequency_df contiene las columnas adicionales "Lematizado" y "Traducción".
print(ingredient_frequency_df)

                               Ingrediente  Frecuencia  \
6                                     salt       59464   
4                                   butter       39850   
36                                   sugar       31794   
56                                   onion       25368   
20                                   water       24416   
...                                    ...         ...   
9551   feta cheese with rosemary and basil           1   
9552                              gai choy           1   
9558                          potato peels           1   
9559                     10 inch pie shell           1   
12720                            nepitella           1   

                                Lematizado                        Traducción  
6                                     salt                               sal  
4                                   butter                           manteca  
36                                   sugar                        

In [51]:
ingredient_frequency_df.head(10)

,Ingrediente,Frecuencia,Lematizado,Traducción
6,salt,59464,salt,sal
4,butter,39850,butter,manteca
36,sugar,31794,sugar,azúcar
56,onion,25368,onion,cebolla
20,water,24416,water,agua
9,eggs,24076,egg,huevo
5,olive oil,22406,olive oil,aceite de oliva
72,flour,20586,flour,harina
10,milk,17984,milk,leche
347,garlic cloves,17126,garlic clove,diente de ajo


In [49]:
ingredient_frequency_df.tail(10)

,Ingrediente,Frecuencia,Lematizado,Traducción
9529,cooked brown and wild rice mix,1,cook brown and wild rice mix,Cocine la mezcla de arroz marrón y salvaje
9533,apple rings,1,apple ring,anillo de manzana
9539,sugar free preserves,1,sugar free preserve,reserva sin azúcar
9540,reduced fat white cake mix,1,reduce fat white cake mix,reducir la mezcla de pastel blanco gordo
9550,roasted walnuts,1,roast walnut,Error de traducción
9551,feta cheese with rosemary and basil,1,feta cheese with rosemary and basil,queso feta con romero y albahaca
9552,gai choy,1,gai choy,gai choy
9558,potato peels,1,potato peel,cáscara de papa
9559,10 inch pie shell,1,10 inch pie shell,Cáscara de pastel de 10 pulgadas
12720,nepitella,1,nepitella,nepitella


Si revisamos ahora la cantidad de ingredientes con la misma palabra, chequeando la columna Lematizado vemos que se reduce la cantidad

In [46]:
palabra_clave = 'onion'
ingredientes_con_palabra_clave = [ingrediente for ingrediente in ingredient_frequency_df['Lematizado'] if palabra_clave in ingrediente]
ingredientes_remove_duplicates = list(set(ingredientes_con_palabra_clave))
print("Cantidad de ingredientes", len(ingredientes_remove_duplicates))

print(ingredientes_remove_duplicates)

Cantidad de ingredientes 141
['tomato with onion and garlic', 'hunt roasted garlic and onion spaghetti sauce', 'spring onion', 'onion hamburger bun', 'dice tomato with sweet onion and roasted garlic', 'french onion soup', 'white pearl onion', 'cheddar french fry onion', 'purple onion', 'onion soup mix', 'sweet onion and pepper relish', 'round onion', 'tomato sauce with onion', 'prepare onion dip', 'onion and garlic crouton', 'onion gravy mix', 'onion in cream sauce', 'green onion dip mix', 'onion powder', 'lipton onion soup mix', 'dice tomato with green pepper celery and onion', 'small onion', 'durkee onion', 'onion and garlic powder', 'onion flavor crouton', 'fresh onion', 'cooking onion', 'garlic and onion tofu', 'garlic and onion pasta sauce', 'granulate onion', 'texas sweet onion', 'red onion ring', 'onion roast garlic soup mix', 'frozen pepper and onion', 'dry onion', 'brown onion', 'sweet red onion', 'french fried onion ring', 'fresh onion ring', 'instant onion', 'cook onion', 'c

Vamos a verificar si hay errores de traduccion en la columna

In [52]:
# Cuenta cuántas filas tienen "Error de traducción" en la columna "Traducción".
cantidad_errores = len(ingredient_frequency_df[ingredient_frequency_df['Traducción'] == 'Error de traducción'])

print("Cantidad de errores de traducción:", cantidad_errores)


Cantidad de errores de traducción: 49


In [53]:
print(ingredient_frequency_df[ingredient_frequency_df['Traducción'] == 'Error de traducción'])

                  Ingrediente  Frecuencia            Lematizado  \
105                  rosemary         983              rosemary   
130                      sage         518                  sage   
404                      pork         252                  pork   
446                      feta         224                  feta   
316                    turkey         219                turkey   
2573                     lamb         130                  lamb   
212                       fat          98                   fat   
1886                     buns          90                   bun   
2244                   savory          64                savory   
5045                  halibut          43               halibut   
2852                     duck          39                  duck   
4419                   rabbit          37                rabbit   
2151               croissants          35             croissant   
6557                 saltines          25               saltin

In [55]:
print(obtener_traduccion('onion'))

cebolla


In [57]:
# Guardar el DataFrame como un archivo JSON.
print(obtener_traduccion('scotch'))

Error de traducción


no hemos consumido la cuota de traduccion, aun asi tenemos algunos ingredientes con errores al traducir. Tambien hay que notar que en algunos casos seria mas conveniente en vez de traducir el ingrediente ya lematizado, deberiamos traducir el ingrediente sin lematizar

A veces tenemos  problemas al traducir, googletrans no está respaldada oficialmente por Google y utiliza la API de forma no oficial, puede ser propensa a cambios y limitaciones no anunciadas. Google podría bloquear o restringir el acceso a esta biblioteca en cualquier momento.

Además, el uso intensivo de esta biblioteca sin autorización o sin cumplir con los términos de servicio de Google Translate podría resultar en bloqueos temporales o permanentes de tu dirección IP. Hay que evaluar  las opciones de suscripción que ofrecen estas API para evitar problemas de límites y bloqueos.

Primeramente las opciones de traduccion en la nube

Google Cloud Translation API:

Proveedor: Google Cloud Platform.
Descripción: Ofrece traducción automática con la misma tecnología que Google Translate. Admite un amplio conjunto de idiomas.
Costo: Ofrecen un nivel gratuito con un límite de caracteres al mes, y luego tarifas por uso adicional.
Documentación y acceso: Google Cloud Translation API.
Microsoft Translator Text API:

Proveedor: Microsoft Azure.
Descripción: Proporciona traducción automática con la tecnología de Microsoft Translator. Admite una amplia variedad de idiomas.
Costo: Ofrece un nivel gratuito con un límite de caracteres al mes y tarifas por uso adicional.
Documentación y acceso: Microsoft Translator Text API.
IBM Language Translator:

Proveedor: IBM Cloud.
Descripción: Ofrece traducción automática con la tecnología de IBM Watson. Admite varios idiomas y tiene capacidades avanzadas de procesamiento de lenguaje natural.
Costo: Tienen un nivel gratuito con un límite de caracteres al mes y tarifas por uso adicional.
Documentación y acceso: IBM Language Translator.
Amazon Translate:

Proveedor: Amazon Web Services (AWS).
Descripción: Proporciona servicios de traducción automática utilizando la tecnología de AWS. Admite varios idiomas y puede integrarse con otros servicios de AWS.
Costo: Tiene una capa gratuita con un límite de caracteres al mes y tarifas por uso adicional.
Documentación y acceso: Amazon Translate

Tendriamos el mismo problema una cuota gratuita y luego un costo por la cantidad de caracteres mensuales a traducir

Revisemos otras opciones de traduccion a través de modelos preentrenados

Existen modelos especificos de traducción sobre temas culinarios  que se pueden  considerar:

CulinaryMT: Es un modelo de traducción automática diseñado específicamente para la industria culinaria. Ofrece una API gratuita que permite traducir hasta 10,000 caracteres al mes. El plan de suscripción ofrece un servicio ampliado: traducción ilimitada con precio por consumo, máxima seguridad de datos y prioridad en la ejecución de peticiones enviadas a la API 1.

GastroNLP: Es un modelo de procesamiento del lenguaje natural (NLP) diseñado para la industria culinaria. Ofrece una API gratuita que permite analizar hasta 5,000 caracteres al mes. El plan de suscripción ofrece un servicio ampliado: análisis ilimitado con precio por consumo, máxima seguridad de datos y prioridad en la ejecución de peticiones enviadas a la API 2.

Culinary Translate: Es un modelo de traducción automática diseñado específicamente para la industria culinaria. Ofrece una API gratuita que permite traducir hasta 10,000 caracteres al mes. El plan de suscripción ofrece un servicio ampliado: traducción ilimitada con precio por consumo, máxima seguridad de datos y prioridad en la ejecución de peticiones enviadas a la API 3.

En estos tenemos el mismo problema del costo de traducción. Hemos revisado la cantidad de caracteres que tenemos que traducir solamente contando la cantidad de caracteres de la columna "Lematizado" de nuestro dataset



Si tuvieramos que traducir la cantidad de caracteres de la columna ingredientes

In [58]:
print(ingredient_frequency_df['Ingrediente'].str.len().sum())

223986


In [60]:
ingredient_frequency_df.to_json('ingredient_translated.json', orient='records')

Un modelo de traducción más avanzado que podemos considerar es MarianMT, desarrollado por Hugging Face. Este modelo es parte de la familia de modelos Marian, que están entrenados específicamente para tareas de traducción. MarianMT admite múltiples idiomas y tiene un rendimiento sólido en la traducción de texto técnico.

Puedes usar el modelo de MarianMT en Python con la biblioteca Transformers de Hugging Face

In [61]:
pip install transformers


Note: you may need to restart the kernel to use updated packages.


In [62]:
import pandas as pd
from transformers import MarianTokenizer, MarianMTModel

# Cargar el modelo y el tokenizador para la traducción de inglés a español
model_name = "Helsinki-NLP/opus-mt-en-es"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# Filtrar los registros con "Error de traducción" en la columna "Traducción"
registros_a_traducir = ingredient_frequency_df   # df[df['Traducción'] == 'Error de traducción']

# Traducir los ingredientes y actualizar la columna "Traducción"
for indice, fila in registros_a_traducir.iterrows():
    ingrediente_lematizado = fila['Ingrediente']
    inputs = tokenizer(ingrediente_lematizado, return_tensors="pt", truncation=True)
    translation = model.generate(**inputs, max_length=50, num_beams=4, length_penalty=0.6, no_repeat_ngram_size=3)
    traduccion = tokenizer.decode(translation[0], skip_special_tokens=True)
    
    # Creemos una columna con la nueva traduccion
    ingredient_frequency_df.at[indice, 'TraduccionMarian'] = traduccion


/opt/conda/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
ingredient_frequency_df.tail(30)

la traducción con este modelo, mejora  a la traduccion anterior en algunos ingredientes poco frecuentes

In [65]:
ingredient_frequency_df.iloc[5500]

Ingrediente                   italian turkey sausage link
Frecuencia                                              7
Lematizado                    italian turkey sausage link
Traducción           Enlace de salchicha de pavo italiano
TraduccionMarian    Enlace italiano de salchichas de pavo
Name: 4989, dtype: object

In [66]:
print("Cantidad de registros sin traducir con MarianModel: ", ingredient_frequency_df['TraduccionMarian'].isna().sum())

Cantidad de registros sin traducir con MarianModel:  0


La traduccion con Marian ademas de haberse hecho con la columna de ingredientes iniciales y no con los lematizados se puede observar que a veces mas clara en algunos ingredientes poco comunes

In [ ]:
ingredient_frequency_df.to_json('ingredient_trans_mariam.json', orient='records')

In [128]:
texto_entrada="""
                Steps:mix together the sour cream , and water pour into a greased 13x9x2-inch pan bake at 350 degrees for 30 minutes cool completely 
               while the cake is baking , prepare the caramel topping combine the flour , salt and baking soda stir into the creamed mixture 
               spread into a greased 9x13 inch baking dish pour the caramel mixture over the batter bake at 350 degrees for 1 hour cool and serve"""

# Tokenizar el texto de entrada
inputs = tokenizer(texto_entrada, return_tensors="pt", truncation=True)

# Generar la traducción
traduccion = model.generate(**inputs, max_length=50, num_beams=4, length_penalty=0.6, no_repeat_ngram_size=3)

# Decodificar la traducción en texto legible
texto_traducido = tokenizer.decode(traduccion[0], skip_special_tokens=True)

# Imprimir la traducción
print("Texto traducido:", texto_traducido)

Texto traducido: Pasos: mezclar la crema agria, y el agua verter en una sartén engrasada de 13x9x2 pulgadas hornear a 350 grados durante 30 minutos enfriar completamente mientras la torta está horneando


In [2]:
df_tr = pd.read_json('/kaggle/input/translate/ingredient_trans_mariam.json')

In [3]:
df_tr.tail(10)

,Ingrediente,Frecuencia,Lematizado,Traducción,TraduccionMarian
12711,cooked brown and wild rice mix,1,cook brown and wild rice mix,Cocine la mezcla de arroz marrón y salvaje,mezcla de arroz integral y integral
12712,apple rings,1,apple ring,anillo de manzana,Anillos de manzana
12713,sugar free preserves,1,sugar free preserve,reserva sin azúcar,confituras sin azúcar
12714,reduced fat white cake mix,1,reduce fat white cake mix,reducir la mezcla de pastel blanco gordo,mezcla de torta blanca de grasa reducida
12715,roasted walnuts,1,roast walnut,Error de traducción,Nueces tostadas
12716,feta cheese with rosemary and basil,1,feta cheese with rosemary and basil,queso feta con romero y albahaca,queso feta con romero y albahaca
12717,gai choy,1,gai choy,gai choy,gai choy
12718,potato peels,1,potato peel,cáscara de papa,cáscaras de patata
12719,10 inch pie shell,1,10 inch pie shell,Cáscara de pastel de 10 pulgadas,Concha de pastel de 10 pulgadas
12720,nepitella,1,nepitella,nepitella,nepitella


In [9]:
print(len(df_tr))

12721


In [8]:
print(len(df_tr['Lematizado'].unique()))

11667


In [ ]:
# Paso 1: Elimina duplicados en el DataFrame original
df_unicos = df.drop_duplicates(subset=['Lematizado'])

In [26]:
palabra_clave = 'onion'
ingredientes_con_palabra_clave = [ingrediente for ingrediente in df_unicos['Lematizado'] if palabra_clave in ingrediente]
ingredientes_remove_duplicates = list(set(ingredientes_con_palabra_clave))
print("Cantidad de ingredientes", len(ingredientes_remove_duplicates))
print(ingredientes_remove_duplicates)

Cantidad de ingredientes 141
['dice tomato with green pepper celery and onion', 'lipton recipe secret onion soup mix', 'red onion strip', 'boiling onion', 'fresh onion', 'onion bread', 'onion soup base', 'lipton golden onion soup mix', 'onion', 'stew tomato with onion and green pepper', 'green onion', 'red spanish onion', 'french fried onion', 'onion paste', 'instant minced onion', 'lipton french onion soup mix', 'cipollini onion', 'beef and onion soup mix', 'mild red onion', 'pickle red onion', 'fry onion', 'frozen mixed potato carrot celery and onion', 'green onion stem', 'onion salt', 'whip onion cream cheese', 'french fried onion ring', 'pickle onion', 'white pearl onion', 'dehydrate onion', 'onion hamburger bun', 'spring onion', 'onion flake', 'low fat soft cheese with chive and onion', 'instant onion', 'frozen chop onion', 'green onion dip mix', 'lipton beef and onion soup mix', 'frozen pearl onion', 'dice onion', 'onion ring batter', 'frenchs cheddar french fry onion', 'yellow o

In [31]:
import pandas as pd

# Supongamos que tienes un DataFrame llamado 'df' con las columnas 'Lematizado', 'Frecuencia', 'Traducción' y 'TraducciónMarian'.

# Paso 1: Agrupar por 'Lematizado' y sumar las frecuencias
ingredientes_unicos_df = df.groupby('Lematizado', as_index=False)['Frecuencia'].sum()

# Paso 2: Obtener las traducciones para cada 'Lematizado'
ingredientes_unicos_df['Traducción'] = df.groupby('Lematizado')['Traducción'].first().values
ingredientes_unicos_df['TraduccionMarian'] = df.groupby('Lematizado')['TraduccionMarian'].first().values

# Paso 3: Ordenar el DataFrame por frecuencia de mayor a menor
ingredientes_unicos_df = ingredientes_unicos_df.sort_values(by='Frecuencia', ascending=False)






In [35]:
ingredientes_unicos_df.tail(30)

,Lematizado,Frecuencia,Traducción,TraduccionMarian
8222,praline sauce,1,salsa de praline,salsa de pralina
8221,praline powder,1,polvo praline,pralina en polvo
2980,dewberry,1,baya de gota,moras
8176,potpourri,1,popurrí,potpourri
2978,devil food cake mix with pudding,1,Mezcla de pastel de comida del diablo con pudín,pastel de alimentos diablos mezclan con pudín
2975,dessert tofu,1,tofu de postre,Tofu de postre
2966,deluxe peanut butter brownie mix,1,Mezcla de brownie de mantequilla de maní de lujo,Mezcla de brownie de mantequilla de maní de lujo
2961,deli maple smoke turkey,1,Deli arce humo pavo,pavo ahumado deli arce
8191,powder egg,1,huevo en polvo,Huevos en polvo
2957,del monte zesty jalapeno pepper dice tomato,1,Del Monte Zesty JalapeNo Pepper Dice Tomato,del monte zesty jalapeno pimiento tomate picado


**Guardamos el dataset lematizado y traducido**

In [ ]:
ingredientes_unicos_df.to_json('ingredient_lema_trans.json', orient='records')

**ESTO FUE UN INTENTO DE AGRUPAR LOS INGREDIETES SINO SE LOGRA A TIEMPO BORRAR DEL NOTEBOOK**

In [36]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans


df = ingredientes_unicos_df
# Supongamos que 'df' es tu DataFrame con los ingredientes en la columna 'Lematizado'.

# Paso 1: Vectorización de los ingredientes utilizando TF-IDF
vectorizer = TfidfVectorizer(max_features=500, stop_words='english')
X = vectorizer.fit_transform(df['Lematizado'])

# Paso 2: Aplicación de k-means
num_clusters = 10  # Define el número de clústeres que deseas crear
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(X)

# Agregar las etiquetas de clústeres a tu DataFrame
df['Cluster'] = kmeans.labels_

# Mostrar los ingredientes en cada clúster
for cluster_id in range(num_clusters):
    print(f"Cluster {cluster_id}:")
    cluster_ingredients = df[df['Cluster'] == cluster_id]['Lematizado']
    print(cluster_ingredients)


/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Cluster 0:
9677                                 sour cream
2673                               cream cheese
5088                                heavy cream
11230                                whip cream
2672                                      cream
                          ...                  
2934                          de fat beef broth
8253         prepare sugar free vanilla pudding
8286     pringle fat free barbecue potato crisp
8210                             powdered cream
8218                          praline ice cream
Name: Lematizado, Length: 632, dtype: object
Cluster 1:
9743                            soy sauce
11559                worcestershire sauce
10574                        tomato sauce
10327                       tabasco sauce
5318                            hot sauce
                       ...               
2984       diana western smokehouse sauce
2983             diana honey garlic sauce
8228     prego ricotta and parmesan sauce
8284                primave